# Split Analysis

This notebook replicate all the Nested Leave-N-Subject-Out splits
(10 outer folds and 5 inner) and sum up some information such as:
1. sets ratios (should be near 0.72 for train, 0.18 for validation, 0.10 for test).
2. class ratios for each sets (should preserve total class ratios).
3. hypothetical baseline classifier's unbalanced accuracy (weighted random guess).


In [ ]:
import glob
import os
import random
import pickle
import warnings
warnings.filterwarnings(
    "ignore", message = "Using padding='same'", category = UserWarning
)

# IMPORT STANDARD PACKAGES
import numpy as np
import pandas as pd

# IMPORT SELFEEG 
import selfeeg
import selfeeg.dataloading as dl

# IMPORT REPOSITORY FUNCTIONS
import sys
sys.path.append('..')
from AllFnc import split
from AllFnc.training import loadEEG

In [ ]:
def check_split2(
    EEGlen: pd.DataFrame,
    EEGsplit: pd.DataFrame,
    Labels=None,
    return_ratio=False,
    verbose=True,
):
    # Check split ratio
    # simply the ratio between the sum of all samples with a specific label set
    # and the sum of all samples with label different from -1
    total_list = EEGsplit[EEGsplit["split_set"] != -1].index.tolist()
    total = EEGlen.iloc[total_list]["N_samples"].sum()
    train_list = EEGsplit[EEGsplit["split_set"] == 0].index.tolist()
    train_ratio = EEGlen.iloc[train_list]["N_samples"].sum() / total
    val_list = EEGsplit[EEGsplit["split_set"] == 1].index.tolist()
    val_ratio = EEGlen.iloc[val_list]["N_samples"].sum() / total
    test_list = EEGsplit[EEGsplit["split_set"] == 2].index.tolist()
    test_ratio = EEGlen.iloc[test_list]["N_samples"].sum() / total

    if verbose:
        print(f"\ntrain ratio:      {train_ratio:.2f}")
        print(f"validation ratio: {val_ratio:.2f}")
        print(f"test ratio:       {test_ratio:.2f}")
    ratios = {
        "train_total": EEGlen.iloc[train_list]["N_samples"].sum(), "train_ratio": train_ratio,
        "val_total": EEGlen.iloc[val_list]["N_samples"].sum(), "val_ratio": val_ratio,
        "test_total": EEGlen.iloc[test_list]["N_samples"].sum(), "test_ratio": test_ratio,
        "class_ratio": None,
        "class_total": None,
    }

    # Check class ratio
    # similar to the previous one but the ratios are calculated with respect to
    # the subset sizes (train test validation sets)
    if Labels is not None:
        Labels = np.array(Labels)
        if len(Labels.shape) != 1:
            raise ValueError("Labels must be a 1d array or a list")
        lab_unique = np.unique(Labels)
        EEGlen2 = EEGlen.copy()  # copy to avoid strange behaviours
        EEGlen2["split_set"] = EEGsplit["split_set"]
        EEGlen2["Labels"] = Labels
        tottrain = EEGlen2.iloc[train_list]["N_samples"].sum()
        totval = EEGlen2.iloc[val_list]["N_samples"].sum()
        tottest = EEGlen2.iloc[test_list]["N_samples"].sum()
        class_ratio = np.zeros((3, len(lab_unique)))
        class_total = np.zeros((3, len(lab_unique)))
        # iterate through train/validation/test sets
        for i in range(3):
            # iterate through each label
            for k in range(len(lab_unique)):
                if i == 0:
                    train_k = EEGlen2.loc[
                        ((EEGlen2["split_set"] == 0) & (EEGlen2["Labels"] == lab_unique[k])),
                        "N_samples",
                    ].sum()
                    class_ratio[i, k] = train_k / tottrain
                    class_total[i, k] = train_k
                elif i == 1:
                    val_k = EEGlen2.loc[
                        ((EEGlen2["split_set"] == 1) & (EEGlen2["Labels"] == lab_unique[k])),
                        "N_samples",
                    ].sum()
                    class_ratio[i, k] = val_k / totval
                    class_total[i, k] = val_k
                else:
                    test_k = EEGlen2.loc[
                        ((EEGlen2["split_set"] == 2) & (EEGlen2["Labels"] == lab_unique[k])),
                        "N_samples",
                    ].sum()
                    class_ratio[i, k] = test_k / tottest
                    class_total[i, k] = test_k
        # print results
        if verbose:
            print(
                f"\ntrain labels ratio:",
                *[f"{lab_unique[k]}={class_ratio[0,k]:.3f}, " for k in range(len(lab_unique))],
            )
            print(
                f"val   labels ratio:",
                *[f"{lab_unique[k]}={class_ratio[1,k]:.3f}, " for k in range(len(lab_unique))],
            )
            print(
                f"test  labels ratio:",
                *[f"{lab_unique[k]}={class_ratio[2,k]:.3f}, " for k in range(len(lab_unique))],
            )
            print("")
        ratios["class_ratio"] = class_ratio
        ratios["class_total"] = class_total
    
    # return calculated ratios if necessary
    if return_ratio:
        return ratios
    else:
        return None

In [ ]:
dataPath       = '/data/delpup/eegpickle/'
pipelineToEval = 'filt'
tasksToEval     = ['alzheimer', 'eyes', 'parkinson', 'motorimagery', 'sleep', 'psychosis']
downsample     = True
z_score        = True
rem_interp     = False
overlap        = 0.0
verbose        = True

summary_table = np.zeros((50*6, 30))
for k, taskToEval in enumerate(tasksToEval):
    if taskToEval.casefold() == 'eyes':
        partition_list = split.create_nested_kfold_subject_split(60,10,5)
    
    elif taskToEval.casefold() == 'sleep':
        partition_list = split.create_nested_kfold_subject_split(71,10,5)
    
    elif taskToEval.casefold() == 'alzheimer':
        # ALZ = subjects 1 to 36; CTL = subjects 37 to 65; FTD = subjects 66 to 88
        part_a = split.create_nested_kfold_subject_split([i for i in range(1,37)], 10, 5)
        part_c = split.create_nested_kfold_subject_split([i for i in range(37,66)], 10, 5)
        part_f = split.create_nested_kfold_subject_split([i for i in range(66,89)], 10, 5)
        partition_list_1 = split.merge_partition_lists(part_a, part_c, 10, 5)
        partition_list = split.merge_partition_lists(partition_list_1, part_f, 10, 5)
    
    elif taskToEval.casefold() == 'parkinson':
        # In this case, two datasets were merged to increase the number 
        # of subjects. So, there are two partition lists to create
    
        #ds003490 - ID 5 - 3Stim
        ctl_id = [i for i in range(28,51)] + [3,5]
        pds_id = [i for i in range(6,28)] + [1,2,4]
        part_c = split.create_nested_kfold_subject_split(ctl_id, 10, 5)
        part_p = split.create_nested_kfold_subject_split(pds_id, 10, 5)
        partition_list_1 = split.merge_partition_lists(part_c, part_p, 10, 5)
    
        #ds002778 - ID 8 - UCSD
        part_c = split.create_nested_kfold_subject_split([i for i in range(1,17)], 10, 5)
        part_p = split.create_nested_kfold_subject_split([i for i in range(17,32)], 10, 5)
        partition_list_2 = split.merge_partition_lists(part_c, part_p, 10, 5)
    
    elif taskToEval.casefold() == 'psychosis':
        # CTL = subjects 101 to 149; PD/PDD/PDMCI = mixing number in [1; 100]
        ctl_id = [1, 3, 4, 9, 10, 12, 13, 14, 17, 19, 21, 22, 24, 25, 27, 29, 30,
                  31, 35, 38, 39, 41, 43, 46, 48, 49, 53, 55, 58, 59]
        psy_id = [2, 5, 6, 7, 8, 11, 15, 16, 18, 20, 23, 26, 28, 32, 33, 34, 36,
                  37, 40, 42, 44, 45, 47, 50, 51, 52, 54, 56, 57, 60, 61]
        part_c = split.create_nested_kfold_subject_split(ctl_id, 10, 5)
        part_p = split.create_nested_kfold_subject_split(psy_id, 10, 5)
        partition_list = split.merge_partition_lists(part_c, part_p, 10, 5)       
    else:
        # three subjects were excluded for the known issue of having 
        # a sampling rate of 128 Hz instead of 160 Hz (and strange trial length).
        subject_list = [i for i in range(1,110) if i not in [88,92,100]] 
        partition_list = split.create_nested_kfold_subject_split(subject_list,10,5)
    
    N_subj = len(partition_list[0][0]) + len(partition_list[0][1]) + len(partition_list[0][2])
    # Section 4: set the training parameters
    
    if dataPath[-1] != os.sep:
        dataPath += os.sep
    if pipelineToEval[-1] != os.sep:
        eegpath = dataPath + pipelineToEval + os.sep
    else:
        eegpath = dataPath + pipelineToEval
    
    if taskToEval.casefold() == 'motorimagery':
        freq = 160
    else:
        freq = 125 if downsample else 250
    
    # Define the partition window length, in second. 4s for every task except the MI
    # which has samples of 4.1s due to the sampling rate
    window = 4.1 if taskToEval.casefold() == 'motorimagery' else 4.0
    
    # Define the number of classes to predict.
    # All tasks are binary except the Alzheimer's one, 
    # which is a multi-class classification (Alzheimer vs FrontoTemporal vs Control)
    if taskToEval.casefold() == 'alzheimer':
        nb_classes = 3
    else:
        nb_classes = 2
    
    # For selfEEG's models instantiation
    Samples = int(freq*window)
    
    # Set the Dataset ID for glob.glob operation in SelfEEG's GetEEGPartitionNumber().
    # It is a single number for every task except for PD that merges two datasets
    if taskToEval.casefold() == 'eyes':
        datasetID = '2'
    elif taskToEval.casefold() == 'alzheimer':
        datasetID = '10'
    elif taskToEval.casefold() == 'motorimagery':
        datasetID = '25'
    elif taskToEval.casefold() == 'sleep':
        datasetID = '20'
    elif taskToEval.casefold() == 'psychosis':
        datasetID = '7'
    else:
        datasetID_1 = '5' # EEG 3-Stim
        datasetID_2 = '8' # UC SD
    
    #  Section 5: Define pytorch's Datasets and dataloaders
    
    loadEEG_args = {
        'return_label': False, 
        'downsample': downsample, 
        'use_only_original': rem_interp,
        'eegsym_train': False,
        'apply_zscore': z_score
    }
    
    if taskToEval.casefold() == 'parkinson':
        glob_input = [datasetID_1 + '_*.pickle', datasetID_2 + '_*.pickle' ]
    else:
        glob_input = [datasetID + '_*.pickle']
    
    # calculate dataset length.
    # Basically it automatically retrieves all the partitions 
    # that can be extracted from each EEG signal
    EEGlen = dl.get_eeg_partition_number(
        eegpath,
        freq,
        window,
        overlap, 
        file_format = glob_input,
        load_function = loadEEG,
        optional_load_fun_args = loadEEG_args,
        includePartial = False if overlap == 0 else True,
        verbose = verbose
    )
    
    # Now we also need to load the labels
    loadEEG_args['return_label'] = True
    
    # Set functions to retrieve dataset, subject, and session from each filename.
    # They will be used by GetEEGSplitTable to perform a subject based split
    dataset_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[0])
    subject_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[1]) 
    session_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[2]) 

    labels = np.zeros(len(EEGlen))
    for i in range(len(EEGlen)):
        path = EEGlen.iloc[i,0]
        with open(path, 'rb') as eegfile:
            EEG = pickle.load(eegfile)
        labels[i] = EEG['label']
    
    for outerFold in range(10):
        for innerFold in range(5):
            
            # fold to eval is the correct index to get the desired train/val/test partition
            foldToEval = outerFold*5 + innerFold

            N_subj_train = len(partition_list[foldToEval][0])
            N_subj_val = len(partition_list[foldToEval][1])
            N_subj_test = len(partition_list[foldToEval][2])
            
            # Now call the GetEEGSplitTable. Since Parkinson task merges two datasets
            # we need to differentiate between this and other tasks
            if taskToEval.casefold() == 'parkinson':
                # Remember
                # 1 --> 5 = EEG 3-Stim  2 --> 8 = UCSD
                train_id   = { 5: partition_list_1[foldToEval][0], 
                               8: partition_list_2[foldToEval][0]}
                val_id     = { 5: partition_list_1[foldToEval][1], 
                               8: partition_list_2[foldToEval][1]}
                test_id    = { 5: partition_list_1[foldToEval][2],
                               8: partition_list_2[foldToEval][2]}
                EEGsplit= dl.get_eeg_split_table(
                    partition_table      = EEGlen,
                    exclude_data_id      = None,  #[8], just checked if UCSD was useful 
                    val_data_id          = val_id,
                    test_data_id         = test_id, 
                    split_tolerance      = 0.001,
                    dataset_id_extractor = dataset_id_ex,
                    subject_id_extractor = subject_id_ex,
                    perseverance         = 10000
                )
                
            else:
                train_id   = partition_list[foldToEval][0]
                val_id     = partition_list[foldToEval][1]
                test_id    = partition_list[foldToEval][2]
                EEGsplit= dl.get_eeg_split_table(
                    partition_table      = EEGlen,
                    exclude_data_id      = None,
                    val_data_id          = val_id,
                    test_data_id         = test_id, 
                    split_tolerance      = 0.001,
                    dataset_id_extractor = subject_id_ex,
                    subject_id_extractor = session_id_ex,
                    perseverance         = 10000
                )
            
            # Define Datasets and preload all data
            trainset = dl.EEGDataset(
                EEGlen, EEGsplit, [freq, window, overlap], 'train', 
                supervised             = True, 
                label_on_load          = True,
                load_function          = loadEEG,
                optional_load_fun_args = loadEEG_args
            )
            
            valset = dl.EEGDataset(
                EEGlen, EEGsplit, [freq, window, overlap], 'validation',
                supervised             = True, 
                label_on_load          = True,
                load_function          = loadEEG,
                optional_load_fun_args = loadEEG_args
            )
            
            testset = dl.EEGDataset(
                EEGlen, EEGsplit, [freq, window, overlap], 'test',
                supervised             = True,
                label_on_load          = True,
                load_function          = loadEEG,
                optional_load_fun_args = loadEEG_args
            )
            ratios = check_split2(EEGlen, EEGsplit, labels, return_ratio = True, verbose = False)
            idx = k*50 + foldToEval
            summary_table[idx, :4] = np.array([N_subj, N_subj_train, N_subj_val, N_subj_test])
            summary_table[idx, 4:8] = np.array([len(trainset)+len(valset)+len(testset) ,
                                           len(trainset), len(valset), len(testset)  ])
            summary_table[idx, 8:11] = np.array([ratios['train_ratio'],
                                            ratios['val_ratio'],
                                            ratios['test_ratio']])
            if taskToEval == 'alzheimer':
                nb = 3
            else:
                nb = 2
                
            summary_table[idx, 11:11+nb] =  ratios['class_total'][0,:]
            summary_table[idx, 14:14+nb] =  ratios['class_ratio'][0,:]
            summary_table[idx, 17:17+nb] =  ratios['class_total'][1,:]
            summary_table[idx, 20:20+nb] =  ratios['class_ratio'][1,:]
            summary_table[idx, 23:23+nb] =  ratios['class_total'][2,:]
            summary_table[idx, 26:26+nb] =  ratios['class_ratio'][2,:]
            summary_table[idx, 29] =  (ratios['class_ratio'][2,:]**2).sum()

In [ ]:
print('Theoretical Baseline - Weighted Random Choice - Accuracy Unbalanced Metric')
print('--------------------------------------------------------------------------')
for i in range(6):
    print(' ')
    if i==0:
        print(' Alzheimer vs FrontoTemporal Dementia vs Control')
    elif i==1:
        print('             Eyes Open Eyes Closed')
    elif i==2:
        print('              Parkinson vs Control')
    elif i==3:
        print('                 Motor Imagery')
    elif i==4:
        print('               Sleep Deprivation')
    elif i==5:
        print('       First Episode Psychosis vs Control')
    m1=np.median(summary_table[50*i:50*(i+1),-1])
    m2=np.percentile(summary_table[50*i:50*(i+1),-1], 75)
    m3=np.percentile(summary_table[50*i:50*(i+1),-1], 25)
    print( ' --------------------------------------------------')
    print( "|  first quartile  |   median   |  third quartile  |")
    print(f"|       {m3:5.3f}      |    {m1:5.3f}   |       {m2:5.3f}      |")
    print( ' --------------------------------------------------')

In [ ]:
summary_table_df = pd.DataFrame(
    summary_table,
    columns=[
        'subj_tot', 'subj_train', 'subj_val', 'subj_test',
        'sample_tot', 'sample_train', 'sample_val', 'sample_test', 
        'ratio_train', 'ratio_val', 'ratio_test',
        'sample_train_class_0', 'sample_train_class_1', 'sample_train_class_2',
        'ratio_train_class_0', 'ratio_train_class_1', 'ratio_train_class_2',
        'sample_val_class_0', 'sample_val_class_1', 'sample_val_class_2', 
        'ratio_val_class_0', 'ratio_val_class_1', 'ratio_val_class_2',
        'sample_test_class_0', 'sample_test_class_1', 'sample_test_class_2', 
        'ratio_test_class_0', 'ratio_test_class_1', 'ratio_test_class_2',
        'baseline'
    ]
)
new_col = [([i]*50) for i in tasksToEval]
new_col = [x for xs in new_col for x in xs]
summary_table_df.insert(loc=0, column='task', value=new_col)
if pipelineToEval in ['raw', 'filt']:
    summary_table_df.to_csv('split_analysis_raw_and_filt.csv', index=False)
else:
    summary_table_df.to_csv('split_analysis_ica_and_isr.csv', index=False)

In [ ]:
if pipelineToEval in ['raw', 'filt']:
    data = pd.read_csv('split_analysis_raw_and_filt.csv')
else:
    data = pd.read_csv('split_analysis_ica_and_isr.csv')

# --------------------------------
# | CHOOSE THE TASK TO SUMMARIZE |
# --------------------------------
task = 'alzheimer'
# --------------------------------

Qlow = 0.05
Qhigh = 0.95
print('------------------------------------------------------')
print('                     TASK:', task.upper())

# get rows with task
data2 = data.loc[data['task']==task].reset_index().drop(columns=['index'])

# class ratio
zero_ratio = data2.iloc[0,[12,18,24]].sum()/data2.iloc[0,5]
one_ratio = data2.iloc[0,[13,19,25]].sum()/data2.iloc[0,5]
two_ratio = data2.iloc[0,[14,20,26]].sum()/data2.iloc[0,5]
print('------------------------------------------------------')
print('CLASS RATIO')
print(f'0 = {zero_ratio:.3f}, 1 = {one_ratio:.3f}, 2 = {two_ratio:.3f}')

# train ratio
md_rt = data2.iloc[:,9].median()
q1_rt = data2.iloc[:,9].quantile(Qlow)
q3_rt = data2.iloc[:,9].quantile(Qhigh)
print('------------------------------------------------------')
print('TRAIN RATIO') 
print(f'median = {md_rt:.3f}, Q_low = {q1_rt:.3f}, Q_high = {q3_rt:.3f}')

# val ratio
md_rt = data2.iloc[:,10].median()
q1_rt = data2.iloc[:,10].quantile(Qlow)
q3_rt = data2.iloc[:,10].quantile(Qhigh)
print('------------------------------------------------------')
print('VALIDATION RATIO')
print(f'median = {md_rt:.3f}, Q_low = {q1_rt:.3f}, Q_high = {q3_rt:.3f}')

# test ratio
md_rt = data2.iloc[:,11].median()
q1_rt = data2.iloc[:,11].quantile(Qlow)
q3_rt = data2.iloc[:,11].quantile(Qhigh)
print('------------------------------------------------------')
print('TEST  RATIO')
print(f'median = {md_rt:.3f}, Q_low = {q1_rt:.3f}, Q_high = {q3_rt:.3f}')

# train class ratio
md_rt0 = data2.iloc[:,15].median()
q1_rt0 = data2.iloc[:,15].quantile(Qlow)
q3_rt0 = data2.iloc[:,15].quantile(Qhigh)
md_rt1 = data2.iloc[:,16].median()
q1_rt1 = data2.iloc[:,16].quantile(Qlow)
q3_rt1 = data2.iloc[:,16].quantile(Qhigh)
md_rt2 = data2.iloc[:,17].median()
q1_rt2 = data2.iloc[:,17].quantile(Qlow)
q3_rt2 = data2.iloc[:,17].quantile(Qhigh)
print('------------------------------------------------------')
print('TRAIN CLASS RATIO')
print(f'CLASS 0: median = {md_rt0:.3f}, Q_low = {q1_rt0:.3f}, Q_high = {q3_rt0:.3f}')
print(f'CLASS 1: median = {md_rt1:.3f}, Q_low = {q1_rt1:.3f}, Q_high = {q3_rt1:.3f}')
print(f'CLASS 2: median = {md_rt2:.3f}, Q_low = {q1_rt2:.3f}, Q_high = {q3_rt2:.3f}')

# val class ratio
md_rt0 = data2.iloc[:,21].median()
q1_rt0 = data2.iloc[:,21].quantile(Qlow)
q3_rt0 = data2.iloc[:,21].quantile(Qhigh)
md_rt1 = data2.iloc[:,22].median()
q1_rt1 = data2.iloc[:,22].quantile(Qlow)
q3_rt1 = data2.iloc[:,22].quantile(Qhigh)
md_rt2 = data2.iloc[:,23].median()
q1_rt2 = data2.iloc[:,23].quantile(Qlow)
q3_rt2 = data2.iloc[:,23].quantile(Qhigh)
print('------------------------------------------------------')
print('VALIDATION CLASS RATIO')
print(f'CLASS 0: median = {md_rt0:.3f}, Q_low = {q1_rt0:.3f}, Q_high = {q3_rt0:.3f}')
print(f'CLASS 1: median = {md_rt1:.3f}, Q_low = {q1_rt1:.3f}, Q_high = {q3_rt1:.3f}')
print(f'CLASS 2: median = {md_rt2:.3f}, Q_low = {q1_rt2:.3f}, Q_high = {q3_rt2:.3f}')

# train class ratio
md_rt0 = data2.iloc[:,27].median()
q1_rt0 = data2.iloc[:,27].quantile(Qlow)
q3_rt0 = data2.iloc[:,27].quantile(Qhigh)
md_rt1 = data2.iloc[:,28].median()
q1_rt1 = data2.iloc[:,28].quantile(Qlow)
q3_rt1 = data2.iloc[:,28].quantile(Qhigh)
md_rt2 = data2.iloc[:,29].median()
q1_rt2 = data2.iloc[:,29].quantile(Qlow)
q3_rt2 = data2.iloc[:,29].quantile(Qhigh)
print('------------------------------------------------------')
print('TEST CLASS RATIO')
print(f'CLASS 0: median = {md_rt0:.3f}, Q_low = {q1_rt0:.3f}, Q_high = {q3_rt0:.3f}')
print(f'CLASS 1: median = {md_rt1:.3f}, Q_low = {q1_rt1:.3f}, Q_high = {q3_rt1:.3f}')
print(f'CLASS 2: median = {md_rt2:.3f}, Q_low = {q1_rt2:.3f}, Q_high = {q3_rt2:.3f}')